In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
from selectolax.parser import HTMLParser

In [3]:
df = pd.read_csv('detailed_stats.csv', header=None, names=['Name', 'Position', 'Comparable', 'Statistics'])

In [4]:
df.head()

,Name,Position,Comparable,Statistics
0,Bradley Barcola,"<div id=""meta"">\n\t\n\n<div class=""media-item""...",[' Att Mid / Wingers'],"{'#stats_standard_dom_lg': '<table class=""stat..."
1,Achraf Hakimi,"<div id=""meta"">\n\t\n\n<div class=""media-item""...",[' Fullbacks'],"{'#stats_standard_dom_lg': '<table class=""stat..."
2,João Neves,"<div id=""meta"">\n\t\n\n<div class=""media-item""...",[' Midfielders'],"{'#stats_standard_dom_lg': '<table class=""stat..."
3,Willian Pacho,"<div id=""meta"">\n\t\n\n<div class=""media-item""...",[' Center Backs'],"{'#stats_standard_dom_lg': '<table class=""stat..."
4,Warren Zaïre-Emery,"<div id=""meta"">\n\t\n\n<div class=""media-item""...","[' Midfielders', ' Fullbacks']","{'#stats_standard_dom_lg': '<table class=""stat..."


In [5]:
df.shape

(20, 4)

In [54]:
df['Name'][9]

'Ousmane Dembélé'

In [55]:
for idx, row in df.iterrows():
    print(row['Name'])

Bradley Barcola
Achraf Hakimi
João Neves
Willian Pacho
Warren Zaïre-Emery
Gianluigi Donnarumma
Marquinhos
Vitinha
Lucas Beraldo
Ousmane Dembélé
Marco Asensio
Lee Kang-in
Fabián Ruiz Peña
Nuno Mendes
Milan Škriniar
Matvei Safonov
Désiré Doué
Randal Kolo Muani
Senny Mayulu
Yoram Zague


In [48]:
pos_html = df['Position'][6]

In [49]:
pos_html

'<div id="meta">\n\t\n\n<div class="media-item"> <img src="https://fbref.com/req/202302030/images/headshots/d5f2f82b_2022.jpg" class="" style="" alt="Marquinhos headshot">\n</div><!-- div.media-item --><div>\n\t<h1>\n\t\t<span>Marquinhos</span>\n\t\t\t</h1>\n\t  <p><strong>Marcos Aoás Corrêa</strong></p>\n\n\n  <p><strong>Position:</strong> DF-MF (CB-FB)&nbsp;▪&nbsp; <strong>Footed:</strong> Right</p>\n  <p><span>183cm</span>,&nbsp;<span>74kg</span>&nbsp;(6-0,&nbsp;165lb)</p>\n  <p>\n  <strong>Born:</strong>\n    <span id="necro-birth" data-birth="1994-05-14">\n    May 14, 1994\n    </span> <span><nobr>(Age:&nbsp;30-207d)</nobr></span>\n\n    \n  <span>\n    \n      in São Paulo, Brazil\n    \n  </span>\n  <span class="f-i" style="background-image:url(\'https://cdn.ssref.net/req/202411012/flags/br-1994.svg\'); ">br</span>\n</p>\n\n  <p><strong>National Team:</strong>&nbsp;<a href="/en/country/BRA/Brazil-Football">Brazil</a> <span class="f-i f-br" style="">br</span></p>\n\n  <p><strong>

In [50]:
import re

text_only = re.sub(r'<[^>]*>', '', pos_html)
text_only = text_only.replace("\n","").replace("\t","").replace("&nbsp;","")
text_only = text_only.split("Wages")[0]

In [51]:
text_only

" Marquinhos  Marcos Aoás Corrêa  Position: DF-MF (CB-FB)▪ Footed: Right  183cm,74kg(6-0,165lb)    Born:        May 14, 1994     (Age:30-207d)                in São Paulo, Brazil        br  National Team:Brazil br  Club: Paris Saint-Germain            Twitter:      @marquinhos_m5           Instagram:  @marquinhosm5      More Player Info// see sr.menus.js:sr_menus_checkInfoCookie to explainfunction sr_menus_checkInfoCookie_inline(browserType) {    var el_info = document.getElementById('info');    var el_button = document.getElementById('meta_more_button');    var bling_len = 2;        if (!el_button || !el_info || !el_info.classList) { console.log('no meta_button'); return; }    var el = el_button;    var siblingsHidden = 0;    while (el = el.previousSibling) { if ((el.nodeType === 1) && (el.offsetWidth "

In [52]:
def extract_player_info(data):
    # Extract the required details using regex
    position = re.search(r'Position:\s*([\w\s\-,()]+)', data)
    footed = re.search(r'Footed:\s*([\w]+)', data)
    height = re.search(r'(\d{3}cm)', data)
    weight = re.search(r'(\d+kg)', data)
    national_team = re.search(r'National Team:\s*([\w\s]+)', data)
    club = re.search(r'Club:\s*([\w\s\-]+)', data)

    return {
        "Position": position.group(1) if position else np.nan,
        "Preferred Foot": footed.group(1) if footed else np.nan,
        "Height": height.group(1) if height else np.nan,
        "Weight": weight.group(1) if weight else np.nan,
        "National Team": national_team.group(1).strip() if national_team else np.nan,
        "Club": club.group(1).strip() if club else np.nan,
    }

In [53]:
extract_player_info(text_only)

{'Position': 'DF-MF (CB-FB)',
 'Preferred Foot': 'Right',
 'Height': '183cm',
 'Weight': '74kg',
 'National Team': 'Brazil br  Club',
 'Club': 'Paris Saint-Germain            Twitter'}

In [12]:
comp = df['Comparable'][4]

In [18]:
comps = []
for pos in eval(comp):
    comps.append(pos.strip())

In [20]:
'|'.join(comps)

'Midfielders|Fullbacks'

In [27]:
df['Name'][10]

'Marco Asensio'

In [22]:
stats_html = eval(df['Statistics'][10])

In [56]:
stats_html

{'#stats_standard_dom_lg': '<table class="stats_table sortable min_width now_sortable sticky_table eq1 eq3 re3 le1" id="stats_standard_dom_lg" data-cols-to-freeze="1,3"> <caption>Standard Stats<span style="color: #666; font-size:smaller">: Domestic Leagues</span> Table</caption> <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup> <thead> <tr class="over_header"> <th aria-label="" data-stat="" colspan="6" class=" over_header center" style="left: -209px; position: sticky; text-align: right; z-index: 20; border-right: 1px solid rgb(148, 150, 152);"></th><th class="not_sticky"></th> <th aria-label="" data-stat="header_playing" colspan="3" class="over_header center group_start not_sticky">Playing Time</th> <th aria-label="" data-stat="header_performance" colspan="8" class="over_header center group_start not_sticky">Performance</th> <th aria-la

In [23]:
stats_html['#stats_standard_dom_lg']

'<table class="stats_table sortable min_width now_sortable sticky_table eq1 eq3 re3 le1" id="stats_standard_dom_lg" data-cols-to-freeze="1,3"> <caption>Standard Stats<span style="color: #666; font-size:smaller">: Domestic Leagues</span> Table</caption> <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup> <thead> <tr class="over_header"> <th aria-label="" data-stat="" colspan="6" class=" over_header center" style="left: -209px; position: sticky; text-align: right; z-index: 20; border-right: 1px solid rgb(148, 150, 152);"></th><th class="not_sticky"></th> <th aria-label="" data-stat="header_playing" colspan="3" class="over_header center group_start not_sticky">Playing Time</th> <th aria-label="" data-stat="header_performance" colspan="8" class="over_header center group_start not_sticky">Performance</th> <th aria-label="" data-stat="header_ex

In [24]:
def process_stats(stats_html):
    table_names = dict()
    for table in stats_html.keys():
        for node in HTMLParser(stats_html[table]).css('caption'):
            text = node.text().split(':')
            table_names[table] = text[0]
    final_dfs = []
    for table in table_names.keys():
        stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
        new_df = pd.DataFrame()
        for col in stats_df.columns:
            if('Unnamed' in col[0]):
                new_df[str(col[1])] = stats_df[col[0]][col[1]]
            else:
                new_df[str(col[0]) + '_' + str(col[1])] = stats_df[col[0]][col[1]]

        stats_df = pd.DataFrame(columns=new_df.columns)
        df_idx = 0
        for idx, row in new_df.iterrows():
            if(row['Season'] is np.nan):
                continue
            if(('Seasons' in row['Season']) or ('Clubs' in row['Squad'])):
                break
            splits = row['Season'].split('-')
            if(len(splits) != 2 or (not splits[0].isnumeric())):
                continue
            stats_df.loc[df_idx] = row
            seasons = row['Season'].split('-')
            stats_df.at[df_idx, 'for_join'] = str(int(seasons[0])) + '-' + str(int(seasons[1])) + '-' + str(df_idx)
            df_idx += 1
        final_dfs.append(stats_df)
    statistics = pd.DataFrame()
    for i in range(len(final_dfs)):
        if i == 0:
            statistics = final_dfs[i].copy()
            continue
        result = pd.concat([statistics, final_dfs[i]], axis=1)
        result = result.loc[:,~result.columns.duplicated()]
        statistics = result.copy()
    statistics.drop(columns=['for_join'], inplace=True)
    return statistics

In [25]:
stats_df = process_stats(stats_html)

C:\Users\shour\AppData\Local\Temp\ipykernel_41624\3081633334.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
C:\Users\shour\AppData\Local\Temp\ipykernel_41624\3081633334.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
C:\Users\shour\AppData\Local\Temp\ipykernel_41624\3081633334.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_df = pd.read_html(stats_html[table], match=table_names[table], flavor='lxml')[0]
C:\Users\shour\AppData\Lo

In [26]:
stats_df

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Performance_Crs,Performance_Int,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,2013-2014,17,Mallorca,es ESP,2. La Liga 2,17th,20,12,1203,13.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-2015,18,Mallorca,es ESP,2. La Liga 2,16th,36,33,2908,32.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-2016,19,Espanyol,es ESP,1. La Liga,13th,34,33,2825,31.4,...,179,16,44,1,0,0,NaN,NaN,NaN,NaN
3,2016-2017,20,Real Madrid,es ESP,1. La Liga,1st,23,11,1082,12.0,...,28,11,22,0,0,0,NaN,NaN,NaN,NaN
4,2017-2018,21,Real Madrid,es ESP,1. La Liga,3rd,32,19,1844,20.5,...,106,9,17,0,0,0,77,3,3,50.0
5,2018-2019,22,Real Madrid,es ESP,1. La Liga,3rd,30,19,1779,19.8,...,78,14,8,3,0,0,76,4,14,22.2
6,2019-2020,23,Real Madrid,es ESP,1. La Liga,1st,9,3,336,3.7,...,11,1,3,0,0,0,16,2,4,33.3
7,2020-2021,24,Real Madrid,es ESP,1. La Liga,2nd,35,21,1869,20.8,...,97,11,6,0,0,0,84,10,7,58.8
8,2021-2022,25,Real Madrid,es ESP,1. La Liga,1st,31,19,1731,19.2,...,71,12,8,0,0,0,83,0,9,0.0
9,2022-2023,26,Real Madrid,es ESP,1. La Liga,2nd,31,15,1442,16.0,...,82,5,10,1,1,0,45,3,2,60.0


In [16]:
final_df = pd.read_csv('statistics.csv')

In [17]:
final_df.head()

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club,Playing Time_MP
0,2010-2011,17,Basel,ch SUI,1. Super Lg,1st,20.0,11.0,1136.0,12.6,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
1,2011-2012,18,Basel,ch SUI,1. Super Lg,1st,24.0,18.0,1595.0,17.7,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
2,2012-2013,19,Gladbach,de GER,1. Bundesliga,8th,22.0,15.0,1477.0,16.4,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
3,2013-2014,20,Gladbach,de GER,1. Bundesliga,6th,28.0,23.0,2023.0,22.5,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
4,2014-2015,21,Gladbach,de GER,1. Bundesliga,3rd,30.0,30.0,2637.0,29.3,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN


In [18]:
df = pd.read_csv('statistics.csv')

In [19]:
df.head()

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club,Playing Time_MP
0,2010-2011,17,Basel,ch SUI,1. Super Lg,1st,20.0,11.0,1136.0,12.6,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
1,2011-2012,18,Basel,ch SUI,1. Super Lg,1st,24.0,18.0,1595.0,17.7,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
2,2012-2013,19,Gladbach,de GER,1. Bundesliga,8th,22.0,15.0,1477.0,16.4,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
3,2013-2014,20,Gladbach,de GER,1. Bundesliga,6th,28.0,23.0,2023.0,22.5,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
4,2014-2015,21,Gladbach,de GER,1. Bundesliga,3rd,30.0,30.0,2637.0,29.3,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN


In [20]:
df

,Season,Age,Squad,Country,Comp,LgRank,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club,Playing Time_MP
0,2010-2011,17,Basel,ch SUI,1. Super Lg,1st,20.0,11.0,1136.0,12.6,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
1,2011-2012,18,Basel,ch SUI,1. Super Lg,1st,24.0,18.0,1595.0,17.7,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
2,2012-2013,19,Gladbach,de GER,1. Bundesliga,8th,22.0,15.0,1477.0,16.4,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
3,2013-2014,20,Gladbach,de GER,1. Bundesliga,6th,28.0,23.0,2023.0,22.5,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
4,2014-2015,21,Gladbach,de GER,1. Bundesliga,3rd,30.0,30.0,2637.0,29.3,...,NaN,Granit Xhaka,MF,Left,185.0,82.0,31.0,NaN,Leverkusen\nWages,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24188,2020-2021,18,Sheffield Utd,eng ENG,1. Premier League,20th,1.0,0.0,1.0,0.0,...,0.0,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN
24189,2021-2022,19,Sheffield Utd,eng ENG,2. Championship,5th,1.0,1.0,66.0,0.7,...,NaN,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN
24190,2021-2022,19,Beerschot,be BEL,1. First Division A,18th,1.0,0.0,8.0,0.1,...,NaN,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN
24191,2022-2023,20,Rochdale,eng ENG,4. League Two,24th,29.0,20.0,1821.0,20.2,...,50.0,Femi Seriki,DF,NaN,NaN,NaN,22.0,NaN,SheffieldUnited,NaN


In [21]:
df['position'].value_counts()

position
DF           5261
MF           4300
FW-MF        3873
DF-MF        3840
FWF          2142
FW-MFF       1597
MFF           863
FW            804
DF-FW-MF      745
DFF           338
DF-MFF        217
DF-FW-MFF     138
DF-FW          49
DF-FWF         24
Name: count, dtype: int64

In [22]:
gk_df = pd.read_csv('gk_statistics.csv')

In [23]:
gk_df['position'].value_counts()

position
GKF    2467
GK      211
Name: count, dtype: int64

In [24]:
gk_df[gk_df['position'] == 'Right']

,Season,Age,Squad,Country,Comp,LgRank,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,...,Aerial Duels_Lost,Aerial Duels_Won%,Name,position,footed,height,weight,age,national_team,current_club


In [26]:
df = pd.read_csv('statistics.csv')
gk_df = pd.read_csv('gk_statistics.csv')

In [27]:
df.shape

(24193, 186)

In [29]:
gk_df['position'].value_counts()

position
GKF    2467
GK      211
Name: count, dtype: int64

In [30]:
gk_df.shape

(2678, 211)